# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import time
import math
from math import radians
import gc


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import time
import math
from math import radians
import gc


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nfl-big-data-bowl-2023/players.csv
/kaggle/input/nfl-big-data-bowl-2023/week6.csv
/kaggle/input/nfl-big-data-bowl-2023/week2.csv
/kaggle/input/nfl-big-data-bowl-2023/pffScoutingData.csv
/kaggle/input/nfl-big-data-bowl-2023/week3.csv
/kaggle/input/nfl-big-data-bowl-2023/week8.csv
/kaggle/input/nfl-big-data-bowl-2023/games.csv
/kaggle/input/nfl-big-data-bowl-2023/week5.csv
/kaggle/input/nfl-big-data-bowl-2023/week7.csv
/kaggle/input/nfl-big-data-bowl-2023/week1.csv
/kaggle/input/nfl-big-data-bowl-2023/week4.csv
/kaggle/input/nfl-big-data-bowl-2023/plays.csv


In [3]:
def get_df_counts(df: pd.DataFrame):
    """get the number of unique counts for each column"""
    total_count = len(df)
    df_unique = pd.DataFrame.from_records([(col, str(df[col].dtype), df[col].count(), total_count-df[col].count(), df[col].nunique()) for col in df.columns],
                                          columns=['column_name', 'dtype', 'non_null_count', 'null_count', 'num_unique'])
    mem = df.memory_usage()
    mem_usage = pd.DataFrame(mem, columns=['usage']).reset_index().rename(columns={'index': 'column_name'})
    mem_usage = mem_usage.assign(usage_mb = mem_usage.usage/1024**2)
    mem_usage.drop(columns=['usage'], inplace=True)
    df_counts = mem_usage.merge(df_unique).sort_values(by='usage_mb', ascending=False)
    
    print(f'memory usage: {sum(mem)/1024**2:.1f}MB')
    return df_counts

def reduce_mem_usage(df, category_limit: int = 128):
    """reduce memory usage"""
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            if not (str(col_type).startswith('int') or str(col_type).startswith('float')):
                continue
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type).startswith('int'):
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int64)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            elif str(col_type).startswith('float'):
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)#
                else:
                    df[col] = df[col].astype(np.float64)
#         else:
#             if df[col].nunique() <= category_limit:
#                 df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print(f'memory usage decreased by {100*(start_mem - end_mem)/start_mem:.1f}, final size {end_mem:.1f} MB')

    return df

In [4]:
def import_data(d):
    df = pd.read_csv(d)
    df = reduce_mem_usage(df)
    return df

In [5]:
def mergeData(df):
    print("Merging data...")
    games = reduce_mem_usage(pd.read_csv("../input/nfl-big-data-bowl-2023/games.csv"))
    plays = reduce_mem_usage(pd.read_csv("../input/nfl-big-data-bowl-2023/plays.csv"))
    pff = reduce_mem_usage(pd.read_csv("../input/nfl-big-data-bowl-2023/pffScoutingData.csv"))
    players = reduce_mem_usage(pd.read_csv("../input/nfl-big-data-bowl-2023/players.csv"))
    players.drop(['height','weight','birthDate', 'collegeName'], axis=1, inplace=True)
    players['nflId'] = players['nflId'].astype(float)
    df = pd.merge(df,games, how = 'left',left_on=['gameId'], right_on=['gameId'])
    df['PlayerTeam'] = np.where(df['team'] == 'home', df['homeTeamAbbr'], df['visitorTeamAbbr'])
    df = df.drop(['homeTeamAbbr','visitorTeamAbbr'], axis=1)
    df = pd.merge(df, plays, how = 'left', left_on = ['gameId', 'playId'], right_on = ['gameId', 'playId'])
    df = pd.merge(df, pff, how = 'left', left_on=['gameId', 'playId', 'nflId'], right_on=['gameId', 'playId', 'nflId'])
    df= pd.merge(df, players, how = 'left', left_on = ['nflId'], right_on = ['nflId'])
    
    print(f'memory: {sum(df.memory_usage())/ 1024**2:.1f}MB')
    return df

In [6]:
def Standardize(D):
    print("Standardizing data...")
    D["ToLeft"] = D['playDirection'] == 'left'
    D['onOffense'] = D.team == D.possessionTeam
    D['absoluteYardlineNumber_std'] = D['absoluteYardlineNumber']
    D.loc[D.ToLeft, 'absoluteYardlineNumber_std'] = 120 - D.absoluteYardlineNumber_std
    #standardize x-coordinate
    D["X_std"] = D.x
    D.loc[D.ToLeft, "X_std"] = 120 - D.loc[D.ToLeft, "x"]
    #standardize y-coordinate
    D["Y_std"] = D.y
    D.loc[D.ToLeft, "Y_std"] = 160/3 - D.loc[D.ToLeft, "y"]
    #standardize orientation in degrees and radians (0 degrees is point straight at opposing end zone)
    D["o_deg"] = D.o
    D["o_deg_std"] = np.mod(D.o-90, 360)
    D.loc[D.ToLeft, "o_deg_std"] = np.mod(D.loc[D.ToLeft, "o_deg_std"] + 180, 360)
    D["o_rad"] = np.mod(D.o, 360) * math.pi/180
    D["o_rad_std"] = np.mod(D.o_rad - math.pi/2, 2*math.pi)
    D.loc[D.ToLeft, "o_rad_std"] = np.mod(D.loc[D.ToLeft, "o_rad_std"] + math.pi, 2 * math.pi)
    #standardize direction in degrees and radians (0 degrees is point straight at opposing end zone)
    D["dir_deg"] = D.dir
    D["dir_deg_std"] = np.mod(D.dir-90, 360)
    D.loc[D.ToLeft, "dir_deg_std"] = np.mod(D.loc[D.ToLeft, "dir_deg_std"] + 180, 360)
    D["dir_rad"] = np.mod(D.dir, 360) * math.pi/180
    D["dir_rad_std"] = np.mod(D.dir_rad - math.pi/2, 2*math.pi)
    D.loc[D.ToLeft, "dir_rad_std"] = np.mod(D.loc[D.ToLeft, "dir_rad_std"] + math.pi, 2 * math.pi)
    D["MPH"] = D.s * 2.23694
    #distance to opposing end zone
    D["end_distance"] = 110 - D.X_std
    # speed in x and y vectors
    D["speed_x"] = round(D.s * np.cos(D.dir_rad_std), 2)
    D["speed_y"] = round(D.s * np.sin(D.dir_rad_std), 2)
    return D

In [7]:
def DistanceLos(D):
    D['dist_los'] = D.X_std - D.absoluteYardlineNumber_std
    return D

In [8]:
def RelativePlayerInfo(D):
    print("Getting relative player info...")
    #qb distance
    qb_frames = D.query('pff_positionLinedUp == "QB"')[['gameId','playId','frameId', 'X_std', 'Y_std','o_rad_std','dir_rad_std']]
    qb_frames = qb_frames.rename(columns = {'X_std':'QB_X', 'Y_std':'QB_Y','o_rad_std': 'QB_o_rad_std','dir_rad_std':'QB_dir_rad_std'})
    D = pd.merge(D, qb_frames, how = 'left', left_on=['gameId', 'playId', 'frameId'], right_on=['gameId', 'playId', 'frameId'])
    D['QB_dx'] = D.X_std-D.QB_X
    D['QB_dy'] = D.Y_std-D.QB_Y
    D['dist_QB'] = np.sqrt(D.QB_dx**2+D.QB_dy**2)
    #football distance    
    football_frames = D.query('team == "football"')[['gameId','playId','frameId', 'X_std', 'Y_std',]]
    football_frames = football_frames.rename(columns = {'X_std':'football_X', 'Y_std':'football_Y'})
    D= pd.merge(D, football_frames, how = 'left', left_on=['gameId', 'playId', 'frameId'], right_on=['gameId', 'playId', 'frameId'])
    D['ball_dx'] = D.X_std-D.football_X
    D['ball_dy'] = D.Y_std-D.football_Y
    D['dist_ball'] = np.sqrt(D.ball_dx**2+D.ball_dy**2)
    #blocked player info
    D_blockingmerge = D[['gameId', 'playId', 'nflId', 'frameId',
        'officialPosition', 'pff_role', 'pff_positionLinedUp','pff_nflIdBlockedPlayer', 'X_std', 'Y_std', 'o_deg_std', 
        'o_rad_std', 'dir_deg_std','dir_rad_std','MPH','speed_y','speed_x']]
    D_blockingmerge = D_blockingmerge.query('nflId == nflId')
    D_blockingmerge.columns = ['blocked_' + str(col) for col in D_blockingmerge.columns]
    D = pd.merge(D, D_blockingmerge, how = 'left', 
                               left_on = ['gameId', 'playId', 'frameId', 'pff_nflIdBlockedPlayer'], 
                               right_on=['blocked_gameId', 'blocked_playId', 'blocked_frameId', 'blocked_nflId'])
    D = D.drop(['blocked_pff_nflIdBlockedPlayer', 'blocked_frameId', 'blocked_nflId', 'blocked_playId', 'blocked_gameId'], axis = 1)
    D['blocked_dx'] = D.X_std - D.blocked_X_std
    D['blocked_dy'] = D.Y_std - D.blocked_Y_std
    D['dist_blocked'] = np.sqrt(D.blocked_dx**2 + D.blocked_dy**2)
    
    
    
    return D
    
    
    
    
    
    
    

In [16]:
# weeks = {1,2,3,4,5,6,7,8}
weeks = range(1,9)

finaldf_list = []
for n in tqdm(weeks):
    start = time.process_time()
    print('Week ' + str(n))
    filename = '../input/nfl-big-data-bowl-2023/week' + str(n) + '.csv'
    Original = import_data(filename)
    print('Merge-------')
    df = mergeData(Original)
    print('Standardize-------')
    Finaldf = Standardize(df)

    Relevantdf = Finaldf[['gameDate', 'gameTimeEastern', 'week','gameId', 'playId', 'frameId', 'nflId', 'displayName',
       'onOffense','team','officialPosition','playDirection', 'x', 'y', 's', 'a', 'dis', 'o', 'dir', 'ToLeft', 'X_std', 'Y_std',
       'o_deg', 'o_deg_std', 'o_rad', 'o_rad_std', 'dir_deg', 'dir_deg_std',
       'dir_rad', 'dir_rad_std', 'MPH','speed_x','speed_y', 'end_distance', 'event',
       'season', 'PlayerTeam',
       'playDescription', 'quarter', 'down', 'yardsToGo', 'possessionTeam',
       'defensiveTeam', 'yardlineSide', 'yardlineNumber', 'gameClock',
       'preSnapHomeScore', 'preSnapVisitorScore', 'passResult', 'penaltyYards',
       'prePenaltyPlayResult', 'playResult', 'foulName1', 'foulNFLId1',
       'foulName2', 'foulNFLId2', 'foulName3', 'foulNFLId3',
       'absoluteYardlineNumber','absoluteYardlineNumber_std', 'offenseFormation', 'personnelO',
       'defendersInBox', 'personnelD', 'dropBackType', 'pff_playAction',
       'pff_passCoverage', 'pff_passCoverageType', 'pff_role',
       'pff_positionLinedUp', 'pff_hit', 'pff_hurry', 'pff_sack',
       'pff_beatenByDefender', 'pff_hitAllowed', 'pff_hurryAllowed',
       'pff_sackAllowed', 'pff_nflIdBlockedPlayer', 'pff_blockType',
       'pff_backFieldBlock' ]]
    
    
    Relevantdf = RelativePlayerInfo(Relevantdf)
    Relevantdf = DistanceLos(Relevantdf)
    Relevantdf = reduce_mem_usage(Relevantdf)
    
    
    
    print(sum(Relevantdf.memory_usage()))
    finaldf_list.append(Relevantdf)
    Relevantdf.to_parquet('FinalData_Week' + str(n) + '.parquet')
    
    gc.collect()
    

  0%|          | 0/1 [00:00<?, ?it/s]

Week 1
memory usage decreased by 34.4, final size 89.6 MB
Merge-------
Merging data...
memory usage decreased by 17.5, final size 0.0 MB
memory usage decreased by 32.0, final size 1.4 MB
memory usage decreased by 33.3, final size 14.4 MB
memory usage decreased by 7.1, final size 0.1 MB
memory: 383.9MB
Standardize-------
Standardizing data...
Getting relative player info...
memory usage decreased by 8.8, final size 573.7 MB
601549636


100%|██████████| 1/1 [01:04<00:00, 64.78s/it]


In [17]:
# Final_1 = pd.read_parquet('/kaggle/working/FinalData_Week1.parquet')

# Final_1.columns[50:]

In [9]:
# week1 = finaldf_list[0]

# week1['dist_los']

In [53]:
# Finaldf = pd.concat(finaldf_list)

# Finaldf = reduce_mem_usage(Finaldf)

# Finaldf.to_parquet('FinalData_2023.parquet')